In [124]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [125]:
import pandas as pd

df = pd.read_json('../../data/News_Category_Dataset_v2.json', lines=True)

In [126]:
df.link[0]

'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89'

In [127]:
article_text = pd.DataFrame()

In [128]:

df2k = df[:2000]
df2k['text'] = text

<ipython-input-128-cd299841938b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2k['text'] = text


In [129]:
len(article_text)

0

In [130]:
from ratelimit import limits, RateLimitException
from backoff import on_exception, expo
import requests

In [131]:
from newspaper.article import ArticleException

In [132]:
df2k = df2k.dropna()

In [133]:
#df2k['text'] = df2k['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#df['text'].head()

df2k['lower_text'] = df2k['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [134]:


#remove punctuation
df2k.lower_text = df2k.lower_text.str.replace('[^\w\s]','')



In [135]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/johnsimmons/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [136]:
#stopword gathering and removal
stop = stopwords.words('english')
df2k['lower_text'] = df2k['lower_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))



In [137]:
df2k.category = df2k.category.str.lower()

In [138]:
y = df2k.category
X = df2k.lower_text

In [139]:
# generate tf-idf vectorization (use sklearn's TfidfVectorizer) for our data
def tfidf(X, y,  stopwords_list): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    vectorizer = TfidfVectorizer(stop_words=stopwords_list)
    tf_idf_train = vectorizer.fit_transform(X_train)
    tf_idf_test = vectorizer.transform(X_test)
    return tf_idf_train, tf_idf_test, y_train, y_test, vectorizer


In [140]:
stopwords_list = stopwords.words('english')
idf_train, idf_test, y_tr, y_t, vectorizer = tfidf(X, y, stopwords_list)

In [141]:
def classify_text(classifier, tf_idf_train, tf_idf_test, y_train):
    classifier.fit(tf_idf_train, y_train)
    train_preds = classifier.predict(tf_idf_train)
    test_preds = classifier.predict(tf_idf_test)
    return train_preds, test_preds

def score_preds(y_test,y_train,test_preds, train_preds):
    print("Train: ", accuracy_score(y_train, train_preds))
    print("Test: ", accuracy_score(y_test, test_preds))
#     confusion_matrix(y_test, test_preds)

In [142]:
rfc = RandomForestClassifier(n_estimators=100)
nb_classifier = MultinomialNB()

In [143]:
nb_train_preds, nb_test_preds = classify_text(nb_classifier, idf_train, idf_test, y_tr)

In [144]:
#Call function for score
score_preds(y_t, y_tr, nb_test_preds, nb_train_preds)

Train:  0.5590230664857531
Test:  0.4959349593495935


In [145]:


#Call function using results of vectorize function using RF model
rf_train_preds, rf_test_preds = classify_text(rfc, idf_train, idf_test, y_tr)

#Call function for score
score_preds(y_t, y_tr, rf_test_preds, rf_train_preds)

Train:  0.9986431478968792
Test:  0.5934959349593496


In [212]:
collected_data_text = []

for idx in df2k[['full_text']].index:
    sub_dict = {'id': idx, 'text': df2k['full_text'][idx]}
    collected_data_text.append(sub_dict)

In [213]:
collected_data = {}

collected_data['text'] = collected_data_text

Updating saved json file


In [14]:
import json

# Load in the json file
file = open('collected_data.json', 'r')
loaded_data = json.load(file)


In [53]:
ld

[]

In [56]:
text = [loaded_data['text'][i]['text'] for i in range(len(loaded_data['text']))]

text_ = []

for i in range(len(loaded_data['text'])):
    dictionary = loaded_data['text'][i]
    t = dictionary['text']
    text_.append(t)